In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19

d:\Anaconda navigator\envs\project\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
model = VGG19(
    include_top = False,
    weights = 'imagenet'
)

model.trainable = False
model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [3]:
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

%pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Note: you may need to restart the kernel to use updated packages.


In [4]:
def load_and_process_image(image_path):
    img = load_img(image_path)
    img = img_to_array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis = 0)
    return img

In [5]:
def deprocess(x):
    # perform the inverse of the preprocessiing step
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]

    x = np.clip(x, 0, 255).astype('uint8')
    return x

def display_image(image):
    if len(image.shape) == 4:
        img = np.squeeze(image, axis = 0)

    img = deprocess(img)

    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    return

In [6]:
import PIL
from PIL import Image

print(PIL.__version__)


11.1.0


In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input



In [8]:
%pip install Pillow

# Ensure CELL INDEX: 3 is executed before running this cell
img = load_and_process_image(r'C:\Users\Lenovo\Downloads\NeuralArt Style Transfer_v\Main\Images\content.png')


Note: you may need to restart the kernel to use updated packages.


In [9]:
# Ensure necessary imports and function definition are included
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np

def load_and_process_image(image_path):
	img = load_img(image_path)
	img = img_to_array(img)
	img = preprocess_input(img)
	img = np.expand_dims(img, axis=0)
	return img

img = load_and_process_image(r'C:\Users\Lenovo\Downloads\NeuralArt Style Transfer_v\Main\Images\content.png')


In [10]:
style_layers = [
    'block1_conv1',
    'block3_conv1',
    'block5_conv1'
]

content_layer = 'block5_conv2'

# intermediate models
content_model = Model(
    inputs = model.input,
    outputs = model.get_layer(content_layer).output
)

style_models = [Model(inputs = model.input,
                      outputs = model.get_layer(layer).output) for layer in style_layers]

In [11]:
# Content Cost
def content_cost(content, generated):
    a_C = content_model(content)
    a_G = content_model(generated)
    cost = tf.reduce_mean(tf.square(a_C - a_G))
    return cost

In [12]:
def gram_matrix(A):
    channels = int(A.shape[-1])
    a = tf.reshape(A, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a = True)
    return gram / tf.cast(n, tf.float32)

In [13]:
lam = 1. / len(style_models)

def style_cost(style, generated):
    J_style = 0

    for style_model in style_models:
        a_S = style_model(style)
        a_G = style_model(generated)
        GS = gram_matrix(a_S)
        GG = gram_matrix(a_G)
        current_cost = tf.reduce_mean(tf.square(GS - GG))
        J_style += current_cost * lam

    return J_style

In [14]:
import time
import tensorflow as tf  # Add this import

generated_images = []

def training_loop(content_path, style_path, iterations=20, a=10., b=20.):
    # Initialise
    content = load_and_process_image(content_path)
    style = load_and_process_image(style_path)
    generated = tf.Variable(content, dtype=tf.float32)

    opt = tf.optimizers.Adam(learning_rate=7.0)

    best_cost = float('inf')
    best_image = None

    start_time = time.time()

    for i in range(iterations):
        with tf.GradientTape() as tape:
            tape.watch(generated)  # Ensure gradients track changes to generated
            J_content = content_cost(content, generated)
            J_style = style_cost(style, generated)
            J_total = a * J_content + b * J_style  # Ensure J_content and J_style are scalars

        grads = tape.gradient(J_total, generated)
        # Fix: Pass generated directly instead of wrapping it in a list
        opt.apply_gradients([(grads, generated)])

        if J_total < best_cost:
            best_cost = J_total
            best_image = generated.numpy()

        if i % (iterations // 10) == 0:
            time_taken = time.time() - start_time
            print(f'Cost at iteration {i}: {J_total.numpy():.4f}. Time elapsed: {time_taken:.2f}s')
            generated_images.append(generated.numpy())

    return best_image

In [ ]:
final = training_loop(
	r'C:\Users\Lenovo\Downloads\NeuralArt Style Transfer_v\Main\Images\content.png',
	r'C:\Users\Lenovo\Downloads\NeuralArt Style Transfer_v\Main\Images\style.png'
)

Cost at iteration 0: 10381280256.0000. Time elapsed: 4.69s
Cost at iteration 2: 4323870720.0000. Time elapsed: 12.77s
Cost at iteration 4: 2762459648.0000. Time elapsed: 26.98s
Cost at iteration 6: 2271082496.0000. Time elapsed: 39.01s
Cost at iteration 8: 1823080192.0000. Time elapsed: 47.37s
Cost at iteration 10: 1460318208.0000. Time elapsed: 56.23s
Cost at iteration 12: 1172826112.0000. Time elapsed: 65.24s
Cost at iteration 14: 984187008.0000. Time elapsed: 73.69s
Cost at iteration 16: 852929152.0000. Time elapsed: 83.28s
Cost at iteration 18: 747446592.0000. Time elapsed: 92.84s


: 

In [12]:
# Ensure CELL INDEX: 14 is executed before running this cell
try:
	# Display the final generated image
	display_image(final)
except NameError:
	print("Error: The variable 'final' is not defined. Please execute the cell that defines 'final' (CELL INDEX: 14) first.")

Error: The variable 'final' is not defined. Please execute the cell that defines 'final' (CELL INDEX: 14) first.
